In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [2]:
Ne = 100
Nx = 3

X = randn(Nx, Ne) .* randn(Nx, Ne)

λ = 0.0
δ = 1e-8
γ = 2.0

poff = 2
pdiag = 1

Cgreedy, ϵgreedy = greedyfit(Nx, poff, pdiag, deepcopy(X), 1, λ, δ, γ)

(Sparse radial map component of dimension 3 and order p = [-1, 2, 1]
, [0.1572457303567653, 0.15003348913568723])

In [41]:
Nx = 5
Ne = 1000

Σ = SymTridiagonal(rand(Nx), 0.05*rand(Nx-1))
@assert isposdef(Symmetric(Matrix(Σ))) "Σ is not positive definite"
Lx = cholesky(Symmetric(inv(Σ))).L
πX = MvNormal(zeros(Nx), Symmetric(Σ))
X = rand(πX, Ne)
Xvalid = rand(πX, Ne)
λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [52]:
@time S = SparseRadialMap(Nx, -1; λ = λ, δ = δ, γ = γ)

  0.000014 seconds (106 allocations: 8.625 KiB)


Sparse Radial Map of dimension Nx = 5 and order p = [[-1], [-1, -1], [-1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1, -1]]
        with parameters (γ, λ, δ, κ) = (2.0, 0.0, 1.0e-8, 10.0)


In [53]:
@time S = optimize(S, X, 2, 0, "kfolds"; verbose = true)

Fold 1:
1 active dimensions  - Training error: 0.2904017679797667, Validation error: 0.2431374846420569
Fold 2:
1 active dimensions  - Training error: 0.26640814074270575, Validation error: 0.3375536593570596
Fold 3:
1 active dimensions  - Training error: 0.26478865566614457, Validation error: 0.34362825558885446
Fold 4:
1 active dimensions  - Training error: 0.3034668250551874, Validation error: 0.19045812436639392
Fold 5:
1 active dimensions  - Training error: 0.2742726477760522, Validation error: 0.3045047164600608
Optimization on the entire data set:
1 active dimensions  - Training error: 0.2802682819827217
Fold 1:
1 active dimensions  - Training error: 0.09976692869752901, Validation error: 0.09786855057218458
2 active dimensions  - Training error: 0.09841735712502349, Validation error: 0.10114844985694359
Fold 2:
1 active dimensions  - Training error: 0.09603808461312713, Validation error: 0.11219889981427227
2 active dimensions  - Training error: 0.09573266278082491, Validation 

Sparse Radial Map of dimension Nx = 5 and order p = [[0], [-1, 0], [-1, 2, 0], [-1, -1, -1, 0], [-1, -1, -1, 2, 0]]
        with parameters (γ, λ, δ, κ) = (2.0, 0.0, 1.0e-8, 10.0)


In [51]:
S[1].coeff

1-element Array{Array{Float64,1},1}:
 [-0.07155850515213437, 1.2457424754907997]

In [29]:
Cgreedy = SparseRadialMapComponent(Nx, -1)
@time Cgreedy, ϵ = optimize(Cgreedy, X, 1, 0, "kfolds", λ, δ, γ; verbose = false)

  0.024131 seconds (480.29 k allocations: 44.274 MiB)


(Sparse radial map component of dimension 5 and order p = [-1, -1, -1, -1, 0]
, [-0.028762879099387236])

In [83]:
@time ForwardDiff.derivative(z-> Cgreedy(vcat(x[1:4], z)), x[5])

  0.272118 seconds (732.85 k allocations: 39.022 MiB, 4.35% gc time)


1.806181259414845

In [86]:
@time ForwardDiff.hessian(z-> Cgreedy(z), x)

  1.250680 seconds (4.98 M allocations: 242.098 MiB, 2.84% gc time)


5×5 Array{Float64,2}:
 0.0   0.0         0.0          0.0          0.0
 0.0  -0.0235844   0.0          0.0          0.0
 0.0   0.0        -0.00810138   0.0          0.0
 0.0   0.0         0.0         -8.58126e-13  0.0
 0.0   0.0         0.0          0.0          0.0